<a href="https://colab.research.google.com/github/jashvidesai/ORF-Thesis/blob/main/ALNSWithSomeOperators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First portion is the initial soln, copied from the previous algorithm file.

In [1]:
import numpy as np
import heapq
import random
from copy import deepcopy

# Problem Parameters
V = range(18)  # Nodes including depot
V_star = range(1, 18)  # Customer nodes
K = range(8)  # Vehicles
Q = [100, 120, 130, 105, 140, 95, 110, 115]  # Vehicle capacities
vehicle_speed = 60  # Speed in km/h
fixed_costs = [120, 138, 118, 122, 110, 110, 115, 113]  # Fixed costs per vehicle

np.random.seed(42)
distances = np.random.randint(10, 51, size=(len(V), len(V)))
for i in V:
    distances[i, i] = 0
    for j in range(i + 1, len(V)):
        distances[j, i] = distances[i, j]
for i in V:
    for j in V:
        for k in V:
            if distances[i, j] > distances[i, k] + distances[k, j]:
                distances[i, j] = distances[i, k] + distances[k, j]

t = (distances / vehicle_speed) * 60  # travel times in minutes
c = distances * 0.093  # travel costs (arbitrary scaling for fuel)

# Delivery and pickup demands
d = [0, 21, 11, 32, 34, 33, 10, 16, 20, 19, 28, 33, 35, 10, 37, 25, 36, 27]  # Delivery demands
p = [0, 23, 7, 22, 29, 35, 7, 28, 10, 26, 27, 6, 31, 6, 30, 21, 37, 13]  # Pickup demands

# Time windows and service times
a = [0, 15, 26, 20, 11, 14, 27, 7, 22, 29, 28, 17, 11, 29, 8, 17, 24, 5]  # Earliest arrival times
b = [300] * 18  # Latest departure times
s = [8, 8, 6, 5, 9, 9, 6, 4, 4, 9, 9, 8, 5, 3, 4, 9, 8, 7]  # Service times

# define a vehicle class to store critical information
class Vehicle:
    def __init__(self, vehicle_id, capacity, speed, cost, depot):
        self.vehicle_id = vehicle_id
        self.capacity = capacity
        self.speed = speed
        self.cost = cost
        self.route = [] # (customer_id, delivered, picked_up, arrival_time)
        self.current_time = 0

        # Updated Capacity Tracking
        self.full_vials = int(self.capacity * 0.75) # Start with 50% full vials
        self.empty_vials = 0 # Initially no empty vials
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials) # Available space
        self.current_location = depot

    def can_add_customer(self, node, travel_time):
        """
        Check if the vehicle can arrive within the time window.
        Returns (boolean) feasibility flag and expected arrival time.
        """
        # check if the vehicle has capacity
        if self.empty_vials == self.capacity:
            return False, self.current_time

        arrival_time = self.current_time + travel_time
        if arrival_time > b[node]:  # Too late
            return False, arrival_time
        if arrival_time < a[node]:  # Arrive early, wait
            arrival_time = a[node]
        return True, arrival_time

    def add_customer(self, node, delivery_demand, pickup_demand, travel_time):
        """
        Assigns a customer to the vehicle while ensuring delivery before pickup
        and allowing split deliveries.
        """
        # Step 1: Check feasibility (time window constraints)
        feasible, arrival_time = self.can_add_customer(node, travel_time)
        if not feasible:
            return False, delivery_demand, pickup_demand

        # Step 2: Deliver first, and update the vehicle load values
        delivered = min(self.full_vials, delivery_demand)
        self.full_vials -= delivered
        delivery_demand -= delivered
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials)

        # Step 3: Pick up second, and update the vehicle load values
        picked_up = min(self.empty_space, pickup_demand)
        self.empty_vials += picked_up
        pickup_demand -= picked_up
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials)

        # Step 4: Save customer in route & update time
        self.route.append((node, delivered, picked_up, arrival_time))
        self.current_time = arrival_time + s[node]
        self.current_location = node

        # ** Debugging Print Statements **
        print(f"Vehicle {self.vehicle_id} visited Customer {node}:")
        print(f"   - Delivered {delivered} (Remaining at customer: {delivery_demand})")
        print(f"   - Picked Up {picked_up} (Remaining at customer: {pickup_demand})")
        print(f"   - Vehicle State: Full Vials = {self.full_vials}, Empty Vials = {self.empty_vials}, Empty Space = {self.empty_space}")


        return True, delivery_demand, pickup_demand

    def return_to_depot(self, depot, t, depot_service_time=30):
        """
        Sends the vehicle back to the depot when full of empty vials.
        Resets load and allows for redeployment.
        """
        travel_time_to_depot = t[self.current_location][depot] # Retrieve from travel time matrix
        self.route.append((depot, 0, 0, self.current_time + travel_time_to_depot)) # Add depot return
        self.current_time += travel_time_to_depot + depot_service_time # Add travel and depot service time

        # Reset vehicle load: Start with 75% full vials, empty vials reset to 0
        self.full_vials = int(self.capacity * 0.75)
        self.empty_vials = 0
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials)

        print(f"Vehicle {self.vehicle_id} returned to depot at time {self.current_time} and reset.")

# visiting order is based on distance to depot
def compute_ordered_route(V_star, depot, distances):
    """
    Computes a heuristic visiting order:
    1. Start with the farthest customer from the depot.
    2. Sequentially add the nearest unvisited customer.
    """
    if not V_star:
        return []

    start_node = max(V_star, key=lambda c: distances[depot][c])  # Start with farthest
    ordered_route = [start_node]
    remaining_nodes = set(V_star) - {start_node}

    while remaining_nodes:
        last_node = ordered_route[-1]
        next_node = min(remaining_nodes, key=lambda c: distances[last_node][c])  # Nearest neighbor
        ordered_route.append(next_node)
        remaining_nodes.remove(next_node)

    return ordered_route

# compute a new ordered route based on proximity to depot (after depot service)
def compute_nearest_ordered_route(V_star, depot, distances):
    """
    Computes a heuristic visiting order:
    1. Start with the closest customer to the depot.
    2. Sequentially add the nearest unvisited customer.
    """
    if not V_star:
        return []

    start_node = min(V_star, key=lambda c: distances[depot][c])  # Start with nearest
    ordered_route = [start_node]
    remaining_nodes = set(V_star) - {start_node}

    while remaining_nodes:
        last_node = ordered_route[-1]
        next_node = min(remaining_nodes, key=lambda c: distances[last_node][c])  # Nearest neighbor
        ordered_route.append(next_node)
        remaining_nodes.remove(next_node)

    return ordered_route

def generate_sequential_solution(V, V_star, K, Q, d, p, a, b, s, t, depot, distances):
    """
    Deploys vehicles sequentially, prioritizing larger vehicles first.
    If a vehicle is redeployed, it follows a new nearest-neighbor order.
    """
    initial_ordered_route = compute_ordered_route(V_star, depot, distances)  # Initial farthest-first order

    # Deploy largest vehicles first
    vehicles = sorted(
        [Vehicle(k, Q[k], vehicle_speed, fixed_costs[k], depot) for k in K],
        key=lambda v: v.capacity,
        reverse=True
    )

    remaining_deliveries = {i: d[i] for i in V_star} # dictionary to track remaining delivery demand
    remaining_pickups = {i: p[i] for i in V_star} # dictionary to track remaining pickup demand

    for vehicle in vehicles:
        print(f"\nDeploying Vehicle {vehicle.vehicle_id} (Capacity {vehicle.capacity})\n")

        current_route = initial_ordered_route[:]

        while True:  # Keep redeploying the vehicle until it can’t serve any more customers
            all_customers_served = True  # Flag to check if all customers are served

            # skips if fully served
            for customer in current_route[:]:
                if remaining_deliveries[customer] == 0 and remaining_pickups[customer] == 0:
                    continue

                success, new_remaining_delivery, new_remaining_pickup = vehicle.add_customer(
                    customer, remaining_deliveries[customer], remaining_pickups[customer],
                    t[vehicle.current_location][customer]
                )

                if not success:
                    break

                remaining_deliveries[customer] = new_remaining_delivery
                remaining_pickups[customer] = new_remaining_pickup
                all_customers_served = False  # At least one customer was served

            vehicle.return_to_depot(depot, t, depot_service_time=30)  # 30-min depot service time

            # Check if there are still unfulfilled customers
            if all_customers_served:
                break  # No point in redeploying, so exit

            # Compute a new order for redeployment based on nearest-first heuristic
            unserved_customers = [i for i in V_star if remaining_deliveries[i] > 0 or remaining_pickups[i] > 0]
            current_route = compute_nearest_ordered_route(unserved_customers, depot, distances)

    return vehicles, remaining_deliveries, remaining_pickups

# Run the updated solution
vehicles_updated, remaining_deliveries, remaining_pickups = generate_sequential_solution(
    V, V_star, K, Q, d, p, a, b, s, t, depot=0, distances=distances
)

print("\n**Final Vehicle Routes and Loads**")
for vehicle in vehicles_updated:
    print(f"Vehicle {vehicle.vehicle_id}: Route {vehicle.route}, Final Load: Full={vehicle.full_vials}, Empty={vehicle.empty_vials}")

print("\n**Final Remaining Deliveries**")
for customer, remaining in remaining_deliveries.items():
    if remaining > 0:
        print(f"Customer {customer}: {remaining} units left to deliver")

print("\n**Final Remaining Pickups**")
for customer, remaining in remaining_pickups.items():
    if remaining > 0:
        print(f"Customer {customer}: {remaining} units left to pick up")


Deploying Vehicle 4 (Capacity 140)

Vehicle 4 visited Customer 5:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 35 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 72, Empty Vials = 35, Empty Space = 33
Vehicle 4 visited Customer 6:
   - Delivered 10 (Remaining at customer: 0)
   - Picked Up 7 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 62, Empty Vials = 42, Empty Space = 36
Vehicle 4 visited Customer 4:
   - Delivered 34 (Remaining at customer: 0)
   - Picked Up 29 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 28, Empty Vials = 71, Empty Space = 41
Vehicle 4 visited Customer 11:
   - Delivered 28 (Remaining at customer: 5)
   - Picked Up 6 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 77, Empty Space = 63
Vehicle 4 visited Customer 9:
   - Delivered 0 (Remaining at customer: 19)
   - Picked Up 26 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 103, Empty Space = 37
Vehic

Removal Operators!

In [2]:
# Random Removal
def random_removal(solution, p, remaining_deliveries, remaining_pickups):
    modified_solution = deepcopy(solution)
    all_customers = set()

    # Extract all customers currently in the solution
    for vehicle in modified_solution:
        for stop in vehicle.route:
            if stop[0] != 0:  # Exclude depot
                all_customers.add(stop[0])

    num_to_remove = int(p * len(all_customers))
    customers_to_remove = random.sample(list(all_customers), min(num_to_remove, len(all_customers)))  # Avoid oversampling

    # Remove customers from their respective routes
    for vehicle in modified_solution:
        vehicle.route = [stop for stop in vehicle.route if stop[0] not in customers_to_remove]

    # Set remaining demand of removed customers to 0 (Prevents reallocation)
    for customer in customers_to_remove:
        remaining_deliveries[customer] = 0
        remaining_pickups[customer] = 0

    return modified_solution, customers_to_remove, remaining_deliveries, remaining_pickups

# Example usage: Apply Random Removal on the generated initial solution
modified_random, removed_random, updated_deliveries, updated_pickups = random_removal(
    vehicles_updated, 0.5, remaining_deliveries, remaining_pickups
)
print("\nRemoved Customers (Random Removal):", removed_random)
print("Updated Remaining Deliveries:", updated_deliveries)
print("Updated Remaining Pickups:", updated_pickups)


Removed Customers (Random Removal): [4, 11, 3, 13, 8, 17, 15, 14]
Updated Remaining Deliveries: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}
Updated Remaining Pickups: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}


In [3]:
# Related Removal
def related_removal(solution, p, distances, remaining_deliveries, remaining_pickups):
    modified_solution = deepcopy(solution)  # Creates a copy of the solution
    all_customers = set()

    # Extract all customers currently in the solution
    for vehicle in modified_solution:
        for stop in vehicle.route:
            if stop[0] != 0:  # Exclude depot
                all_customers.add(stop[0])

    if not all_customers:
        return modified_solution, [], remaining_deliveries, remaining_pickups

    num_to_remove = int(p * len(all_customers))
    seed_customer = random.choice(list(all_customers))  # Randomly select a seed customer

    # Compute distances from the seed customer to all others and sort
    related_customers = sorted(all_customers, key=lambda c: distances[seed_customer][c])
    customers_to_remove = related_customers[:min(num_to_remove, len(related_customers))]  # Avoid oversampling

    # Remove selected customers from all routes
    for vehicle in modified_solution:
        vehicle.route = [stop for stop in vehicle.route if stop[0] not in customers_to_remove]

    # Set remaining demand of removed customers to 0 (Prevents reallocation)
    for customer in customers_to_remove:
        remaining_deliveries[customer] = 0
        remaining_pickups[customer] = 0

    return modified_solution, customers_to_remove, remaining_deliveries, remaining_pickups

# Example Usage
modified_related, removed_related, updated_deliveries, updated_pickups = related_removal(
    vehicles_updated, 0.5, distances, remaining_deliveries, remaining_pickups
)
print("\nRemoved Customers (Related Removal):", removed_related)
print("Updated Remaining Deliveries:", updated_deliveries)
print("Updated Remaining Pickups:", updated_pickups)


Removed Customers (Related Removal): [12, 1, 9, 15, 4, 10, 8, 2]
Updated Remaining Deliveries: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}
Updated Remaining Pickups: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}


In [4]:
# Worst Removal
def worst_removal(solution, p, distances, fixed_costs, remaining_deliveries, remaining_pickups):
    modified_solution = deepcopy(solution)
    all_customers = set()
    customer_total_cost = {}

    # Extract all customers and calculate cost impact
    for vehicle in modified_solution:
        for i, stop in enumerate(vehicle.route):
            if stop[0] != 0:  # Exclude depot
                all_customers.add(stop[0])

                # Get previous and next stops
                prev_stop = vehicle.route[i - 1] if i > 0 else (0, 0, 0, 0)
                next_stop = vehicle.route[i + 1] if i < len(vehicle.route) - 1 else (0, 0, 0, 0)  # Depot if last stop

                # Compute cost impact: added travel distance and vehicle fixed cost
                cost = (
                    distances[prev_stop[0]][stop[0]] +
                    distances[stop[0]][next_stop[0]] -
                    distances[prev_stop[0]][next_stop[0]] +
                    fixed_costs[vehicle.vehicle_id]
                )

                # Aggregate total cost contribution per customer
                customer_total_cost[stop[0]] = customer_total_cost.get(stop[0], 0) + cost

    if not all_customers:
        return modified_solution, [], remaining_deliveries, remaining_pickups

    num_to_remove = int(p * len(all_customers))

    # Sort customers by total cost contribution in descending order
    sorted_customers = sorted(customer_total_cost.items(), key=lambda x: x[1], reverse=True)

    # Extract the highest-cost customers for removal
    customers_to_remove = [customer for customer, _ in sorted_customers[:min(num_to_remove, len(sorted_customers))]]

    # Remove selected customers from all routes
    for vehicle in modified_solution:
        vehicle.route = [stop for stop in vehicle.route if stop[0] not in customers_to_remove]

    # Set remaining demand of removed customers to 0 (Prevents reallocation)
    for customer in customers_to_remove:
        remaining_deliveries[customer] = 0
        remaining_pickups[customer] = 0

    return modified_solution, customers_to_remove, remaining_deliveries, remaining_pickups

# Example Usage
modified_worst, removed_worst, updated_deliveries, updated_pickups = worst_removal(
    vehicles_updated, 0.5, distances, fixed_costs, remaining_deliveries, remaining_pickups
)

print("\nRemoved Customers (Worst Removal):", removed_worst)
print("Updated Remaining Deliveries:", updated_deliveries)
print("Updated Remaining Pickups:", updated_pickups)


Removed Customers (Worst Removal): [10, 14, 15, 3, 12, 11, 13, 7]
Updated Remaining Deliveries: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}
Updated Remaining Pickups: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}


Insertion Operators!

In [64]:
# Parallel Insertion
def parallel_insertion(solution, removed_customers, c, t, vehicle_capacities, remaining_deliveries, remaining_pickups, a, b, s):
    # Priority queue for selecting the best insertion based on lowest cost
    insertion_heap = []
    retry_customers = set()

    # Evaluate all removed customers across all vehicles and available routes
    for customer in removed_customers:
        for vehicle in solution:
            for idx in range(len(vehicle.route) - 1):  # Iterate over current route positions

                # Extract current route stops
                prev_stop = vehicle.route[idx][0]  # Previous stop
                next_stop = vehicle.route[idx + 1][0]  # Next stop
                prev_arrival_time = vehicle.route[idx][3]  # Arrival time at previous stop

                # Compute travel time to the customer
                travel_time_to_customer = t[prev_stop][customer]
                arrival_time_customer = prev_arrival_time + travel_time_to_customer

                # Enforce time window constraints
                if arrival_time_customer > b[customer]:  # Arrives too late
                    continue
                elif arrival_time_customer < a[customer]:  # Arrives too early, must wait
                    arrival_time_customer = a[customer]  # Adjust arrival time

                # Compute insertion cost
                insertion_cost = (
                    c[prev_stop][customer] +
                    c[customer][next_stop] -
                    c[prev_stop][next_stop]
                )

                # Compute max feasible delivery and pickup within vehicle capacity constraints
                max_delivery = min(remaining_deliveries.get(customer, 0), vehicle.full_vials)
                max_pickup = min(remaining_pickups.get(customer, 0), vehicle.empty_space)

                # If the vehicle can take any amount of the order, add to priority queue
                if max_delivery > 0 or max_pickup > 0:
                    heapq.heappush(insertion_heap, (insertion_cost, idx, vehicle.vehicle_id, vehicle, customer, max_delivery, max_pickup))

    # Process insertions in order of lowest cost
    while insertion_heap:
        _, _, _, vehicle, customer, delivery, pickup = heapq.heappop(insertion_heap)

        # Skip if customer has already been fully served
        if remaining_deliveries.get(customer, 0) == 0 and remaining_pickups.get(customer, 0) == 0:
            continue

        # Assign feasible demand and update vehicle load
        assigned_delivery = min(remaining_deliveries[customer], vehicle.full_vials)
        assigned_pickup = min(remaining_pickups[customer], vehicle.empty_space)

        vehicle.full_vials -= assigned_delivery
        vehicle.empty_vials += assigned_pickup
        vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

        # Insert customer into route at the selected position
        vehicle.route.insert(idx + 1, (customer, assigned_delivery, assigned_pickup, vehicle.current_time))
        # Ensure sequential time values for future stops
        for i in range(idx + 2, len(vehicle.route)):
            prev_stop = vehicle.route[i - 1]
            next_stop = vehicle.route[i][0]
            travel_time = t[prev_stop[0]][next_stop]

            # Ensure no time jumps
            arrival_time = max(prev_stop[3] + travel_time, a[next_stop])
            vehicle.route[i] = (next_stop, vehicle.route[i][1], vehicle.route[i][2], arrival_time + s[next_stop])

        # Update remaining demand
        remaining_deliveries[customer] -= assigned_delivery
        remaining_pickups[customer] -= assigned_pickup

        # Remove customer from removed_customers if fully served
        if remaining_deliveries[customer] == 0 and remaining_pickups[customer] == 0:
            removed_customers.remove(customer)

        # If customer demand is not fully met, add them back to the retry queue
        if remaining_deliveries[customer] > 0 or remaining_pickups[customer] > 0:
            retry_customers.add(customer)

    # Retry unfulfilled customers using available vehicles
    if retry_customers:
        print(f"Retrying insertion for customers: {retry_customers}\n")
        return parallel_insertion(solution, retry_customers, c, t, vehicle_capacities, remaining_deliveries, remaining_pickups, a, b, s)

    # Handle Unused Vehicles Using Nearest Neighbor Heuristic
    unused_vehicles = [v for v in solution if len(v.route) == 1] # Only depot
    unserved_customers = [cust for cust in remaining_deliveries if remaining_deliveries[cust] > 0 or remaining_pickups[cust] > 0]

    if unused_vehicles and unserved_customers:
        print("Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...")

        # Sort unused vehicles by largest capacity
        unused_vehicles.sort(key=lambda v: v.capacity, reverse=True)

        for vehicle in unused_vehicles:
            if not unserved_customers:
                break  # Exit if all demand is met

            vehicle.route = [(0, 0, 0, 0)]  # Start from depot
            vehicle.current_time = 0

            while True:
                feasible_customers = [
                    c for c in unserved_customers
                    if vehicle.current_time + t[vehicle.route[-1][0]][c] <= b[c]
                    and (remaining_deliveries[c] > 0 or remaining_pickups[c] > 0)
                ]

                if not feasible_customers:
                    break  # No more feasible customers for this vehicle

                next_customer = max(
                    feasible_customers,
                    key=lambda c: (-t[vehicle.route[-1][0]][c], remaining_deliveries[c] + remaining_pickups[c])
                )

                travel_time = t[vehicle.route[-1][0]][next_customer]
                arrival_time = max(vehicle.current_time + travel_time, a[next_customer])

                # Assign feasible delivery and pickup
                assigned_delivery = min(remaining_deliveries[next_customer], vehicle.full_vials)
                assigned_pickup = min(remaining_pickups[next_customer], vehicle.empty_space)

                # Skip if nothing to deliver/pick up
                if assigned_delivery == 0 and assigned_pickup == 0:
                    break  # Let another vehicle handle it later

                # Update vehicle load
                vehicle.full_vials -= assigned_delivery
                vehicle.empty_vials += assigned_pickup
                vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

                vehicle.route.append((next_customer, assigned_delivery, assigned_pickup, round(arrival_time, 2)))
                vehicle.current_time = round(arrival_time + s[next_customer], 2)

                # Update demand
                remaining_deliveries[next_customer] -= assigned_delivery
                remaining_pickups[next_customer] -= assigned_pickup

                if remaining_deliveries[next_customer] == 0 and remaining_pickups[next_customer] == 0:
                    unserved_customers.remove(next_customer)

                # Stop if vehicle is full or out of time
                if vehicle.empty_space == 0 or vehicle.current_time > max(b):
                    break

            # Return to depot if any customers were served
            if len(vehicle.route) > 1:
                return_time = t[vehicle.route[-1][0]][0]
                vehicle.route.append((0, 0, 0, round(vehicle.current_time + return_time, 2)))


    return solution, remaining_deliveries, remaining_pickups

# Example Usage: Reinsert Customers Removed by Random Removal
initial_deliveries = {i: d[i] for i in V_star}  # Convert delivery list to dictionary
initial_pickups = {i: p[i] for i in V_star}  # Convert pickup list to dictionary

modified_random, removed_random, updated_deliveries, updated_pickups = random_removal(
    deepcopy(vehicles_updated), 0.5, deepcopy(initial_deliveries), deepcopy(initial_pickups)
)

updated_solution, final_deliveries, final_pickups = parallel_insertion(
    modified_random, removed_random, c, t, Q, initial_deliveries, initial_pickups, a, b, s
)

# Final Check
print("\n**Demand Satisfaction Check**")
unsatisfied_deliveries = {cust: amt for cust, amt in final_deliveries.items() if amt > 0}
unsatisfied_pickups = {cust: amt for cust, amt in final_pickups.items() if amt > 0}

if not unsatisfied_deliveries and not unsatisfied_pickups:
    print("All customer demands are satisfied!")
else:
    print("Some customer demands remain unsatisfied!")
    if unsatisfied_deliveries:
        print("Remaining Delivery Demand:", unsatisfied_deliveries)
    if unsatisfied_pickups:
        print("Remaining Pickup Demand:", unsatisfied_pickups)

# --- Print Results ---
print("\n**Updated Routes After Random Removal + Parallel Insertion**")
for vehicle in updated_solution:
    print(f"Vehicle {vehicle.vehicle_id}: Route {[(c, d, p, float(t)) for (c, d, p, t) in vehicle.route]}")

Retrying insertion for customers: {13}

Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...

**Demand Satisfaction Check**
All customer demands are satisfied!

**Updated Routes After Random Removal + Parallel Insertion**
Vehicle 4: Route [(5, 33, 35, 37.0), (11, 28, 6, 97.0), (9, 0, 26, 116.0), (7, 0, 21, 170.0), (0, 0, 0, 196.0), (16, 36, 37, 237.0), (7, 16, 7, 256.0), (0, 0, 0, 323.0), (13, 0, 0, 407.0), (13, 0, 0, 410.0), (13, 0, 0, 413.0), (13, 0, 0, 416.0), (13, 9, 6, 419.0), (2, 11, 7, 451.0), (1, 21, 23, 470.0), (8, 20, 10, 509.0), (6, 10, 7, 529.0), (4, 34, 29, 548.0), (0, 0, 0, 584.0)]
Vehicle 2: Route [(11, 5, 0, 24.0), (9, 19, 0, 43.0), (12, 22, 31, 138.0), (3, 0, 22, 198.0), (10, 0, 27, 227.0), (14, 0, 6, 261.0), (0, 0, 0, 277.0), (0, 0, 0, 319.0), (15, 25, 21, 349.0)]
Vehicle 1: Route [(12, 13, 0, 32.0), (3, 32, 0, 92.0), (10, 20, 0, 121.0), (14, 0, 24, 155.0), (0, 0, 0, 171.0), (14, 37, 0, 201.0), (10, 8, 0, 230.0), (0, 0, 0, 264.0), (13, 